In [29]:

from models_otimizador import otimizador
import pandas as pd
import json
import pulp

import logging

# Configurar o logging para mostrar mensagens INFO
# permitir uso de acentos no formato
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()  # Para mostrar no output do notebook
    ],
    style='%'
)

# Ou de forma mais simples:
logging.getLogger().setLevel(logging.INFO)


%load_ext autoreload
%reload_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Resumo dos resultados

In [30]:
# Importar resultados das runs de saidas/saidas.json
with open('saidas/saidas.json', 'r') as f:
    resultados = json.load(f)
    
# Transformar em DataFrame para melhor visualização em que cada run é uma linha
df_resultados = pd.DataFrame(resultados).T
# Transformar as keys dos valores da coluna options em novas colunas
options_df = df_resultados['options'].apply(pd.Series)
# substituir coluna options pelas colunas expandidas
df_resultados = pd.concat([df_resultados.drop(columns=['options']), options_df], axis=1)
# fazer o mesmo com os valores da coluna custos
custos_df = df_resultados['custos'].apply(pd.Series)
df_resultados = pd.concat([df_resultados.drop(columns=['custos']), custos_df], axis=1)

# Formatar os valores numéricos com separador de milhar
pd.options.display.float_format = '{:,.2f}'.format

In [31]:
df_resultados

,status,solution_time,objective_value,NOME,NO_BOG,BOG_CTE,BOG_VAR,LOCK_POLICY,CAPCOST,CAPCOST_VALUE,...,CAPCOST_version,TIMELIMIT,RUN,custo_fixo,custo_spot,custo_lcp,custo_demurrage,custo_top,custo_cap,custo_total
1,Solução ótima encontrada,0.97,"1,049,465,920.60",Cen1,True,False,False,False,False,NaN,...,V1,10800,1.00,"118,374,400.00","270,804,480.60","618,856,000.00","41,431,040.00",0.00,0.00,"1,049,465,920.60"
2,Solução ótima encontrada,9.12,"1,069,757,920.60",Cen1,False,True,False,False,False,NaN,...,NaN,10800,2.00,"118,374,400.00","291,096,480.60","618,856,000.00","41,431,040.00",0.00,0.00,"1,069,757,920.60"
3,Solução ótima encontrada,9.91,"1,075,635,634.01",Cen1,False,False,True,False,False,NaN,...,NaN,10800,3.00,"118,374,400.00","296,974,194.01","618,856,000.00","41,431,040.00",0.00,0.00,"1,075,635,634.01"
4,Solução ótima encontrada,0.67,"1,069,757,920.60",Cen1,False,True,False,True,False,NaN,...,NaN,7200,4.00,"118,374,400.00","291,096,480.60","618,856,000.00","41,431,040.00",0.00,0.00,"1,069,757,920.60"
5,Solução ótima encontrada,0.32,"1,075,835,554.72",Cen1,False,False,True,True,False,NaN,...,NaN,7200,5.00,"118,374,400.00","297,174,114.72","618,856,000.00","41,431,040.00",0.00,0.00,"1,075,835,554.72"
6,Solução ótima encontrada,1.01,"1,055,089,640.78",Cen1,True,False,False,False,True,NaN,...,V1,10800,6.00,"118,374,400.00","270,804,480.60","618,856,000.00","41,431,040.00",0.00,NaN,NaN
7,Solução ótima encontrada,0.39,"1,055,404,266.62",Cen1,True,False,False,True,True,NaN,...,V1,10800,7.00,"118,374,400.00","270,804,480.60","618,856,000.00","41,431,040.00",0.00,NaN,NaN
8,Solução ótima encontrada,1.05,"1,058,041,039.80",Cen1,True,False,False,False,True,NaN,...,V1,10800,8.00,"118,374,400.00","270,804,480.60","618,856,000.00","41,431,040.00",0.00,NaN,NaN
9,Solução ótima encontrada,0.60,"1,058,520,785.24",Cen1,True,False,False,True,True,NaN,...,V1,10800,9.00,"118,374,400.00","270,804,480.60","618,856,000.00","41,431,040.00",0.00,NaN,NaN
10,Solução ótima encontrada,5.47,"1,073,748,547.06",Cen1,False,True,False,False,True,0.21,...,NaN,7200,NaN,"118,374,400.00","291,096,480.60","618,856,000.00","41,431,040.00",0.00,"3,990,626.46","1,073,748,547.06"
